In [9]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from config import login


In [10]:
links = pd.read_csv("movies_csv/links.csv")
movies = pd.read_csv("movies_csv/movies.csv")
ratings = pd.read_csv("movies_csv/ratings.csv")
tags = pd.read_csv("movies_csv/tags.csv")